In [1]:
import requests
import pprint
import json
import mysql.connector

state_code = "FL"


url_all_hospitals = f"http://www.communitybenefitinsight.org/api/get_hospitals.php?"
url_hospitals_per_state = f"http://www.communitybenefitinsight.org/api/get_hospitals.php?state={state_code}"

payload = {}
headers = {
    'Cookie': 'PHPSESSID=nb9s8neh28dr9uuol0eg517l2m'
}


#hospitals for florida
response_hospitals_per_state  = requests.request("GET", url_hospitals_per_state, headers=headers, data=payload)

#all Hospitals
response_all_hospitals = requests.request("GET", url_all_hospitals, headers=headers, data=payload)

# Save the JSON object to a file
with open(f"hospital_data_per_state_{state_code}.json", "w") as f:
    json.dump(json.loads(response_hospitals_per_state.content), f, indent=4)

p = pprint.PrettyPrinter(indent=4)
data_all_hosp = json.loads(response_all_hospitals.content)
data_state_hosp = json.loads(response_hospitals_per_state.content)
#p.pprint(json.loads(response_hospitals_per_state.content))

Get hospitals per ID

In [2]:
hospital_id = "2164"

url_hospital_data_per_id = f"http://www.communitybenefitinsight.org/api/get_hospital_data.php?hospital_id={hospital_id}"

responce_hospital_data_per_id = requests.request("GET", url_hospital_data_per_id, headers=headers, data=payload)

# Save the JSON object to a file
with open(f"hospital_data_per_id_{hospital_id}.json", "w") as f:
    json.dump(json.loads(responce_hospital_data_per_id.content), f, indent=4)

#p.pprint(json.loads(responce_hospital_data_per_id.content))

In [3]:
#data = json.loads(json_string)
#print((data[0]['name']))

#print all ids where 'name' contains 'state'


#for i in range(len(data)):
    #if  'state' in (data[i]['name']).lower():
        #print(data[i]['hospital_id'])

In [4]:
#connect to mysql with env. file

from dotenv import load_dotenv
import os
import mysql.connector
from mysql.connector import Error

load_dotenv('personal_data.env')
db_name = "mydb2"

try: 
    connection = mysql.connector.connect(
        host = os.getenv('DB_HOST'),
        user = os.getenv('DB_USER'),
        password = os.getenv('DB_PASSWORD')
    )

    mycursor = connection.cursor()
    #creating a new database instead of connecting to the existing one
    mycursor.execute(f"create database IF NOT EXISTS {db_name}")
    print(f"Database {db_name} created successfully.")

except Error as e:
    print(f"Error: {e}")


Database mydb2 created successfully.


In [17]:
#selecting created database

mycursor.execute(f"USE {db_name}")

#dynamically create the table based on the first hospital keys

if data_state_hosp:
    first_hosp = data_state_hosp[0]
    column_definitions = []
    for column_name in first_hosp.keys():
        #WE ARE ASSUMING ALL DATA IS IN STRING FORMAT FOR SIMPLICITY
        column_definitions.append(f"{column_name} VARCHAR(255)")



    #creating a table
    table_name = "stateHospitals"
    mycursor.execute(f"create table if not exists {table_name} ({' ,'.join(column_definitions)})")

#insert all state data into the table
for hospital in data_state_hosp:
    columns = hospital.keys()
    values = []
    for value in hospital.values():
        values.append(value)


    insert_query = f"insert into {table_name} ({', '.join(columns)}) values ({', '.join(['%s']*len(values))})"
    mycursor.execute(insert_query, list(values))

    connection.commit()